<center>
<img src="https://habrastorage.org/files/fd4/502/43d/fd450243dd604b81b9713213a247aa20.jpg" />
</center> 
     
## <center>  [mlcourse.ai](https://mlcourse.ai) – Open Machine Learning Course 

#### <center> Author: [Yury Kashnitsky](https://yorko.github.io) (@yorko) 

# <center>Assignment #2. Fall 2019
## <center> Part 2. Gradient boosting

**In this assignment, you're asked to beat a baseline in the ["Flight delays" competition](https://www.kaggle.com/c/flight-delays-fall-2018).**

This time we decided to share a pretty decent CatBoost baseline, you'll have to improve the provided solution.

Prior to working on the assignment, you'd better check out the corresponding course material:
 1. [Classification, Decision Trees and k Nearest Neighbors](https://nbviewer.jupyter.org/github/Yorko/mlcourse_open/blob/master/jupyter_english/topic03_decision_trees_kNN/topic3_decision_trees_kNN.ipynb?flush_cache=true), the same as an interactive web-based [Kaggle Kernel](https://www.kaggle.com/kashnitsky/topic-3-decision-trees-and-knn) 
 2. Ensembles:
  - [Bagging](https://nbviewer.jupyter.org/github/Yorko/mlcourse_open/blob/master/jupyter_english/topic05_ensembles_random_forests/topic5_part1_bagging.ipynb?flush_cache=true), the same as a [Kaggle Kernel](https://www.kaggle.com/kashnitsky/topic-5-ensembles-part-1-bagging)
  - [Random Forest](https://nbviewer.jupyter.org/github/Yorko/mlcourse_open/blob/master/jupyter_english/topic05_ensembles_random_forests/topic5_part2_random_forest.ipynb?flush_cache=true), the same as a [Kaggle Kernel](https://www.kaggle.com/kashnitsky/topic-5-ensembles-part-2-random-forest)
  - [Feature Importance](https://nbviewer.jupyter.org/github/Yorko/mlcourse_open/blob/master/jupyter_english/topic05_ensembles_random_forests/topic5_part3_feature_importance.ipynb?flush_cache=true), the same as a [Kaggle Kernel](https://www.kaggle.com/kashnitsky/topic-5-ensembles-part-3-feature-importance)
 3. - [Gradient boosting](https://nbviewer.jupyter.org/github/Yorko/mlcourse_open/blob/master/jupyter_english/topic10_boosting/topic10_gradient_boosting.ipynb?flush_cache=true), the same as a [Kaggle Kernel](https://www.kaggle.com/kashnitsky/topic-10-gradient-boosting) 
   - Logistic regression, Random Forest, and LightGBM in the "Kaggle Forest Cover Type Prediction" competition: [Kernel](https://www.kaggle.com/kashnitsky/topic-10-practice-with-logit-rf-and-lightgbm) 
 4. You can also practice with demo assignments, which are simpler and already shared with solutions:
  - "Decision trees with a toy task and the UCI Adult dataset": [assignment](https://www.kaggle.com/kashnitsky/a3-demo-decision-trees) + [solution](https://www.kaggle.com/kashnitsky/a3-demo-decision-trees-solution)
  - "Logistic Regression and Random Forest in the credit scoring problem": [assignment](https://www.kaggle.com/kashnitsky/assignment-5-logit-and-rf-for-credit-scoring) + [solution](https://www.kaggle.com/kashnitsky/a5-demo-logit-and-rf-for-credit-scoring-sol)
 5. There are also 7 video lectures on trees, forests, boosting and their applications: [mlcourse.ai/video](https://mlcourse.ai/video) 
 6. mlcourse.ai tutorials on [categorical feature encoding](https://www.kaggle.com/waydeherman/tutorial-categorical-encoding) (by Wayde Herman) and [CatBoost](https://www.kaggle.com/mitribunskiy/tutorial-catboost-overview) (by Mikhail Tribunskiy)
 7. Last but not the least: [Public Kernels](https://www.kaggle.com/c/flight-delays-fall-2018/notebooks) in this competition

### Your task is to:
 1. beat **"A2 baseline (10 credits)"** on Public LB (**0.75914** LB score)
 2. rename your [team](https://www.kaggle.com/c/flight-delays-fall-2018/team) in full accordance with A1 and the [course rating](https://docs.google.com/spreadsheets/d/15e1K0tg5ponA5R6YQkZfihrShTDLAKf5qeKaoVCiuhQ/) (to appear on 16.09.2019)
 
This task is intended to be relatively easy. Here you are not required to upload your reproducible solution.
 
### <center> Deadline for A2: 2019 October 6, 20:59 CET (London time)

In [1]:
import warnings
import numpy as np
import pandas as pd
from pathlib import Path
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from catboost import CatBoostClassifier

**Read the data**

In [2]:
# PATH_TO_DATA = Path('../input/flight-delays-fall-2018/')
PATH_TO_DATA = Path('./data/')

In [3]:
train_df = pd.read_csv(PATH_TO_DATA / 'flight_delays_train.csv')

In [4]:
train_df.head()

,Month,DayofMonth,DayOfWeek,DepTime,UniqueCarrier,Origin,Dest,Distance,dep_delayed_15min
0,c-8,c-21,c-7,1934,AA,ATL,DFW,732,N
1,c-4,c-20,c-3,1548,US,PIT,MCO,834,N
2,c-9,c-2,c-5,1422,XE,RDU,CLE,416,N
3,c-11,c-25,c-6,1015,OO,DEN,MEM,872,N
4,c-10,c-7,c-6,1828,WN,MDW,OMA,423,Y


In [5]:
test_df = pd.read_csv(PATH_TO_DATA / 'flight_delays_test.csv')

In [6]:
test_df.head()

,Month,DayofMonth,DayOfWeek,DepTime,UniqueCarrier,Origin,Dest,Distance
0,c-7,c-25,c-3,615,YV,MRY,PHX,598
1,c-4,c-17,c-2,739,WN,LAS,HOU,1235
2,c-12,c-2,c-7,651,MQ,GSP,ORD,577
3,c-3,c-25,c-7,1614,WN,BWI,MHT,377
4,c-6,c-6,c-3,1505,UA,ORD,STL,258


**Create only one feature - “flight” (this you need to improve - add more features)**

In [7]:
train_df = pd.read_csv(PATH_TO_DATA / 'flight_delays_train.csv')
test_df = pd.read_csv(PATH_TO_DATA / 'flight_delays_test.csv')

train_df['flight'] = train_df['Origin'] + '-->' + train_df['Dest']
test_df['flight'] = test_df['Origin'] + '-->' + test_df['Dest']

train_df['Dep_hour'] =  train_df['DepTime']//100
train_df['Dep_minute'] =  train_df['DepTime']%100

# train_df['OnTime'] = train_df['Dep_minute']%5 == 0    # Плохо работает
# train_df['OnTime'] = train_df['OnTime'].astype('object')

train_df['Dep_minute'] = train_df['Dep_minute'].astype('object')
train_df['Dep_hour'].replace(to_replace=[24,25], value=0, inplace=True)
train_df['Dep_hour'] = train_df['Dep_hour'].astype('object')
# train_df.drop(['DepTime'], inplace=True, axis=1)

test_df['Dep_hour'] =  test_df['DepTime']//100
test_df['Dep_minute'] =  test_df['DepTime']%100
test_df['Dep_minute'] = test_df['Dep_minute'].astype('object')
test_df['Dep_hour'].replace(to_replace=[24,25], value=0, inplace=True)
test_df['Dep_hour'] = test_df['Dep_hour'].astype('object')
# test_df.drop(['DepTime'], inplace=True, axis=1)
# 
# train_df['Month'] = train_df['Month'].apply(lambda x: x.split('-')[1])
# train_df['DayofMonth'] = train_df['DayofMonth'].apply(lambda x: x.split('-')[1])
# train_df['DayOfWeek'] = train_df['DayOfWeek'].apply(lambda x: x.split('-')[1])
# 
# test_df['Month'] = test_df['Month'].apply(lambda x: x.split('-')[1])
# test_df['DayofMonth'] = test_df['DayofMonth'].apply(lambda x: x.split('-')[1])
# test_df['DayOfWeek'] = test_df['DayOfWeek'].apply(lambda x: x.split('-')[1])

train_df.head()

,Month,DayofMonth,DayOfWeek,DepTime,UniqueCarrier,Origin,Dest,Distance,dep_delayed_15min,flight,Dep_hour,Dep_minute
0,c-8,c-21,c-7,1934,AA,ATL,DFW,732,N,ATL-->DFW,19,34
1,c-4,c-20,c-3,1548,US,PIT,MCO,834,N,PIT-->MCO,15,48
2,c-9,c-2,c-5,1422,XE,RDU,CLE,416,N,RDU-->CLE,14,22
3,c-11,c-25,c-6,1015,OO,DEN,MEM,872,N,DEN-->MEM,10,15
4,c-10,c-7,c-6,1828,WN,MDW,OMA,423,Y,MDW-->OMA,18,28


In [8]:
train_df.dtypes

Month                object
DayofMonth           object
DayOfWeek            object
DepTime               int64
UniqueCarrier        object
Origin               object
Dest                 object
Distance              int64
dep_delayed_15min    object
flight               object
Dep_hour             object
Dep_minute           object
dtype: object

In [9]:
def seasons(month):
    if month in ['12', '1', '2']:
        return 'winter'
    elif month in ['3', '4', '5']:
        return 'spring'
    elif month in ['6', '7', '8']:
        return 'summer'
    elif month in ['9', '10', '11']:
        return 'autumn'

train_df['Season'] = train_df['Month'].apply(lambda x: x.split('-')[1]).apply(seasons)#.astype('object')
test_df['Season'] = test_df['Month'].apply(lambda x: x.split('-')[1]).apply(seasons)

# train_df.drop(['Month', 'DayofMonth'], axis=1, inplace=True)
# test_df.drop(['Month', 'DayofMonth'], axis=1, inplace=True)
test_df.head()

,Month,DayofMonth,DayOfWeek,DepTime,UniqueCarrier,Origin,Dest,Distance,flight,Dep_hour,Dep_minute,Season
0,c-7,c-25,c-3,615,YV,MRY,PHX,598,MRY-->PHX,6,15,summer
1,c-4,c-17,c-2,739,WN,LAS,HOU,1235,LAS-->HOU,7,39,spring
2,c-12,c-2,c-7,651,MQ,GSP,ORD,577,GSP-->ORD,6,51,winter
3,c-3,c-25,c-7,1614,WN,BWI,MHT,377,BWI-->MHT,16,14,spring
4,c-6,c-6,c-3,1505,UA,ORD,STL,258,ORD-->STL,15,5,summer


In [10]:
def weekend(day):
    if day in ['6', '7']:
        return 'weekend'
    else:
        return 'working day'

train_df['Weekend'] = train_df['DayOfWeek'].apply(lambda x: x.split('-')[1]).apply(weekend)
test_df['Weekend'] = test_df['DayOfWeek'].apply(lambda x: x.split('-')[1]).apply(weekend)

# train_df.drop(['DayOfWeek'], axis=1, inplace=True)
# test_df.drop(['DayOfWeek'], axis=1, inplace=True)

test_df.head()

,Month,DayofMonth,DayOfWeek,DepTime,UniqueCarrier,Origin,Dest,Distance,flight,Dep_hour,Dep_minute,Season,Weekend
0,c-7,c-25,c-3,615,YV,MRY,PHX,598,MRY-->PHX,6,15,summer,working day
1,c-4,c-17,c-2,739,WN,LAS,HOU,1235,LAS-->HOU,7,39,spring,working day
2,c-12,c-2,c-7,651,MQ,GSP,ORD,577,GSP-->ORD,6,51,winter,weekend
3,c-3,c-25,c-7,1614,WN,BWI,MHT,377,BWI-->MHT,16,14,spring,weekend
4,c-6,c-6,c-3,1505,UA,ORD,STL,258,ORD-->STL,15,5,summer,working day


In [11]:
def part_of_day(hour):
    if int(hour) in range(6,12,1):
        return 'morning'
    if int(hour) in range(12,18,1):
        return 'day'
    if int(hour) in range(18, 24):
        return 'evening'
    else:
        return 'night'

train_df['PartOfTheDay'] = train_df['Dep_hour'].apply(part_of_day).astype('object')
test_df['PartOfTheDay'] = test_df['Dep_hour'].apply(part_of_day).astype('object')
# 
# train_df.drop(['Dep_hour', 'Dep_minute', 'Origin', 'Dest'], axis=1, inplace=True)
# test_df.drop(['Dep_hour', 'Dep_minute', 'Origin', 'Dest'], axis=1, inplace=True)

test_df.head()

,Month,DayofMonth,DayOfWeek,DepTime,UniqueCarrier,Origin,Dest,Distance,flight,Dep_hour,Dep_minute,Season,Weekend,PartOfTheDay
0,c-7,c-25,c-3,615,YV,MRY,PHX,598,MRY-->PHX,6,15,summer,working day,morning
1,c-4,c-17,c-2,739,WN,LAS,HOU,1235,LAS-->HOU,7,39,spring,working day,morning
2,c-12,c-2,c-7,651,MQ,GSP,ORD,577,GSP-->ORD,6,51,winter,weekend,morning
3,c-3,c-25,c-7,1614,WN,BWI,MHT,377,BWI-->MHT,16,14,spring,weekend,day
4,c-6,c-6,c-3,1505,UA,ORD,STL,258,ORD-->STL,15,5,summer,working day,day


In [12]:
def distance_cat(distance):
    if int(distance) < 500:
        return 'near'
    if 500 <= int(distance) < 1000:
        return 'middle'
    if int(distance) >= 1000:
        return 'far'
    
train_df['Distance'].mean()    
# test_df['DistCat'] = test_df['Distance'].apply(distance_cat)
# train_df['DistCat'] = train_df['Distance'].apply(distance_cat)

729.39716

**Remember indexes of categorical features (to be passed to CatBoost)**

In [13]:
categ_feat_idx = np.where(train_df.drop('dep_delayed_15min', axis=1).dtypes == 'object')[0]
categ_feat_idx

array([ 0,  1,  2,  4,  5,  6,  8,  9, 10, 11, 12, 13], dtype=int64)

In [14]:
train_df.drop('dep_delayed_15min', axis=1).columns[categ_feat_idx]

Index(['Month', 'DayofMonth', 'DayOfWeek', 'UniqueCarrier', 'Origin', 'Dest',
       'flight', 'Dep_hour', 'Dep_minute', 'Season', 'Weekend',
       'PartOfTheDay'],
      dtype='object')

**Allocate a hold-out set (a.k.a. a validation set) to validate the model**

In [15]:
X_train = train_df.drop('dep_delayed_15min', axis=1).values
y_train = train_df['dep_delayed_15min'].map({'Y': 1, 'N': 0}).values
X_test = test_df.values

In [16]:
X_train_part, X_valid, y_train_part, y_valid = train_test_split(X_train, y_train, 
                                                                test_size=0.3, 
                                                                random_state=17)

**Train Catboost with default arguments, passing only the indexes of categorical features.**

In [17]:
ctb = CatBoostClassifier(random_seed=17, silent=True)

In [18]:
%%time
ctb.fit(X_train_part, y_train_part,
        cat_features=categ_feat_idx);

Wall time: 2min 26s


In [19]:
ctb_valid_pred = ctb.predict_proba(X_valid)[:, 1]

**We got some 0.756 ROC AUC on the hold-out set.**

In [20]:
roc_auc_score(y_valid, ctb_valid_pred)

0.7832731668562037

**Train on the whole train set, make prediction on the test set. We got ~0.734 in the competition - "Catboost starter" baseline**

In [74]:
%%time
ctb.fit(X_train, y_train,
        cat_features=categ_feat_idx);

Wall time: 3min 50s


In [75]:
ctb_test_pred = ctb.predict_proba(X_test)[:, 1]

In [76]:
with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    
    sample_sub = pd.read_csv(PATH_TO_DATA / 'sample_submission.csv', 
                             index_col='id')
    sample_sub['dep_delayed_15min'] = ctb_test_pred
    sample_sub.to_csv('ctb_pred.csv')

In [24]:
!head ctb_pred.csv

"head" ­Ґ пў«пҐвбп ў­гваҐ­­Ґ© Ё«Ё ў­Ґи­Ґ©
Є®¬ ­¤®©, ЁбЇ®«­пҐ¬®© Їа®Ја ¬¬®© Ё«Ё Ї ЄҐв­л¬ д ©«®¬.


Now's your turn! Go and improve the model to beat **"A2 baseline (10 credits)"** - **0.75914** LB score. It's crucial to come up with some good features. 

For discussions, stick to the **#a2_kaggle_fall2019** thread in the **mlcourse_ai_news** [ODS Slack](http://opendatascience.slack.com) channel. Serhii Romanenko (@serhii_romanenko) will be there to help. 

Welcome to Kaggle!

<img src='https://habrastorage.org/webt/fs/42/ms/fs42ms0r7qsoj-da4x7yfntwrbq.jpeg' width=50%>
*from the ["Nerd Laughing Loud"](https://www.kaggle.com/general/76963) thread.*